# Euchre Simulation Code

In [1]:
import random
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

The Euchre games are simulated by creating a large dataframe of 24 rows. Each row represents a different card from the playable deck. The dataframe is randomly shuffled and split into 4 individual dataframes that contain 5 rows each (representing the 5 cards each player gets). The 4 dataframes represent the 4 Euchre players: north, south, east, and west. In Euchre, south means the 1st person perspective. In this case, that's me. North is south's (my) partner. East and West are the opponents that sit to south's right and left. 

In the simulation process, dataframes get created that evaluate how good a hand is for each suit. The card game is played using the 4 individual dataframes that will get manipulated using the functions defined below. 

# Create functions that will be used in creating Euchre hand simulations

In [2]:
# This function creates columns that indicate if a card is a trump card, a Left Bower, or a Right Bower for each of the 4 suits
def create_indicators(row):
    strength = ''
    bower = ''
    trump = ''
    strength = row['Strength']
    if row['Strength'] == 13:
        bower = 'Right'
    elif row['Strength'] == 12:
        bower = 'Left'
    if row['Strength'] >= 7:
        trump = 'Y'
    else:
        trump = 'N'
    return strength, bower, trump

# Create function that alters the Left Bower as a trump card for each corresponding suit
def alter_suits(suits):
    for suit in suits:
        alt_suit = []
        for index, row in deck.iterrows():
            if row[f'{suit}_Bower'] == 'Left':
                alt_suit.append(suit)
            else:
                alt_suit.append(row['Suit'])
        deck[f'{suit}_Suit'] = alt_suit

# Create an order based off who the first dealer is in the simulation. Each player takes their turn in a clockwise order
def create_order(dealer):
    if dealer == 'north':
        return ['east', 'south', 'west', 'north']
    elif dealer == 'east':
        return ['south', 'west', 'north', 'east']
    elif dealer == 'south':
        return ['west', 'north', 'east', 'south']
    else:
        return ['north', 'east', 'south', 'west']

# Function that counts the number of suits in hand
def count_suits(players, suits):
    for player in players:
        for suit in suits:
            df = globals()[player]
            suit_count_list = []
            for value in df[f'{suit}_Suit']:
                suit_count_list.append((df[f'{suit}_Suit']==value).sum())
            df[f'{suit}_suit_count'] = suit_count_list
            globals()[player] = df

# Function that recounts potential suits based on if the dealer picks up the turned up card
def recount_suits(dealer, suits):
    for suit in suits:
        df = globals()[dealer]
        suit_count_list = []
        for value in df[f'{suit}_Suit']:
            suit_count_list.append((df[f'{suit}_Suit']==value).sum())
        df[f'{suit}_suit_count'] = suit_count_list
        globals()[dealer] = df

# Function that determines how good a suit would be for the dealer if they picked up the turned up card
def picked_up_scenario(dealer_df, trump):
    if (dealer_df['ID'] == turned_up['ID'].values[0]).sum()>0:
        dealer_df = dealer_df
    else:
        low = min(dealer_df[f'{trump}_Strength'].values)
        discard = dealer_df.loc[dealer_df[f'{trump}_Strength'] == low]
        discard = discard.sample(frac=1)[0:1]
        dealer_df = dealer_df.loc[~(dealer_df['ID']==discard['ID'].values[0])]
        return pd.concat([dealer_df, turned_up])

# Build evaluation dataframe that will be used to summarize each suit's values if the suit gets selected as trump
def build_evaluation(players, suit):
    for player in players:
        player_df = globals()[player]
        suit_list = []
        rb_list = []
        lb_list = []
        TAce_list = []
        TKing_list = []
        TQueen_list = []
        TTen_list = []
        TNine_list = []
        Ace_count_list = []
        King_count_list = []
        Queen_count_list = []
        Jack_count_list = []
        Ten_count_list = []
        Nine_count_list = []
        Trump_count_list = []
        strength_list = []
        non_trump_suit_count = []
        non_trump_turned_down_color_ind = []
        start_order_list = []
        dealer_ind_list = []
        max_strength_list = []

        for suit in suits:
            if (suit == turned_up['Suit'].values[0]) & (player == dealer):
                df_to_evaluate = player_df.copy()
                picked_up_evaluate = picked_up_scenario(df_to_evaluate, turned_up['Suit'].values[0])
                player_df = picked_up_evaluate
            else:
                player_df = globals()[player]
            suit_list.append(suit)
            if (player_df[f'{suit}_Strength']==13).sum() > 0:
                rb_list.append('Y')
            else:
                rb_list.append('N')
            if (player_df[f'{suit}_Strength']==12).sum() > 0:
                lb_list.append('Y')
            else:
                lb_list.append('N')
            if (player_df[f'{suit}_Strength']==11).sum() > 0:
                TAce_list.append('Y')
            else:
                TAce_list.append('N')
            if (player_df[f'{suit}_Strength']==10).sum() > 0:
                TKing_list.append('Y')
            else:
                TKing_list.append('N')
            if (player_df[f'{suit}_Strength']==9).sum() > 0:
                TQueen_list.append('Y')
            else:
                TQueen_list.append('N')
            if (player_df[f'{suit}_Strength']==8).sum() > 0:
                TTen_list.append('Y')
            else:
                TTen_list.append('N')
            if (player_df[f'{suit}_Strength']==7).sum() > 0:
                TNine_list.append('Y')
            else:
                TNine_list.append('N')
            Ace_count_list.append((player_df[f'{suit}_Strength']==6).sum())
            King_count_list.append((player_df[f'{suit}_Strength']==5).sum())
            Queen_count_list.append((player_df[f'{suit}_Strength']==4).sum())
            Jack_count_list.append((player_df[f'{suit}_Strength']==3).sum())
            Ten_count_list.append((player_df[f'{suit}_Strength']==2).sum())
            Nine_count_list.append((player_df[f'{suit}_Strength']==1).sum())
            Trump_count_list.append((player_df[f'{suit}_Trump']=='Y').sum())
            strength_list.append(player_df[f'{suit}_Strength'].sum())
            non_trump_suit_count.append(player_df[f'{suit}_Suit'].nunique()-player_df.loc[player_df[f'{suit}_Suit']==suit][f'{suit}_Suit'].nunique())
            if (suit != turned_up['Suit'].values[0]) \
            and (deck.loc[deck['Suit']==suit]['Color'][0:1].values == turned_up['Color'].values[0]):
                non_trump_turned_down_color_ind.append('Y')
            else:
                non_trump_turned_down_color_ind.append('N')
            start_order_list.append(int(order.index(player))+1)
            if player == dealer:
                dealer_ind_list.append('Y')
            else:
                dealer_ind_list.append('N')
            max_strength_list.append(max(player_df[f'{suit}_Strength']))

        summary_data = {
            'Suit': suit_list,
            'Right_Bower': rb_list,
            'Left_Bower': lb_list,
            'TAce': TAce_list,
            'TKing': TKing_list,
            'TQueen': TQueen_list,
            'TTen': TTen_list,
            'TNine': TNine_list,
            'Ace_count': Ace_count_list,
            'King_count': King_count_list,
            'Queen_count': Queen_count_list,
            'Jack_count': Jack_count_list,
            'Ten_count': Ten_count_list,
            'Nine_count': Nine_count_list,
            'Count_Trump': Trump_count_list,
            'Strength': strength_list,
            'Non_Trump_Suit_Count': non_trump_suit_count,
            'Non_Trump_Turned_Down_Color_Ind': non_trump_turned_down_color_ind,
            'Start_Order': start_order_list,
            'Dealer': dealer_ind_list,
            'Strongest_Card': max_strength_list
        }
        summary_df = pd.DataFrame(summary_data)

        naming = f'{player}_summary'
        globals()[naming] = summary_df

# Determines what card to discard if the dealer picks up the turned up card
def pick_it_up(dealer, trump):
    dealer_df = globals()[dealer]
    if (dealer_df['ID'] == turned_up['ID'].values[0]).sum()>0:
        dealer_df = dealer_df
    else:
        if len(dealer_df.loc[(dealer_df[f'{trump}_suit_count'] == 1) & (dealer_df[f'{trump}_Strength'] <= 4)]) > 0:
            dealer_df = dealer_df.loc[(dealer_df[f'{trump}_suit_count'] == 1) & (dealer_df[f'{trump}_Strength'] <= 4)]
        low = min(dealer_df[f'{trump}_Strength'])
        discard = dealer_df.loc[dealer_df[f'{trump}_Strength'] == low]
        discard = discard.sort_values(f'{trump}_Strength', ascending = True).sort_values(f'{trump}_suit_count', ascending = True)[0:1]
        dealer_df = globals()[dealer]
        dealer_df = dealer_df.loc[dealer_df['ID']!=discard['ID'].values[0]]
        dealer_df = pd.concat([dealer_df, turned_up])
        globals()[dealer]=dealer_df

# Determines which suit is the best to bid trump on and makes a predicted call based on general Euchre rules for ordering trump or going alone
def south_decide_trump(south):
    global trump
    global caller
    global alone_ind
    global south_call
    global seen_cards
    south = globals()[south]
    seen_cards = pd.DataFrame(columns = south.columns.tolist())
    alone_ind = 'N'
    trump = 'none'
    caller = 'none'
    turned_up_suit = turned_up['Suit'].values[0]
    naming = 'south_summary'
    summary_df = globals()[naming]
    summary_df = summary_df.loc[~(summary_df['Suit']==turned_up_suit)]
    summary_df = summary_df.loc[((summary_df['Count_Trump'] >= 2) & (summary_df['Strongest_Card'] == 13)) | (summary_df['Count_Trump'] >= 3)]
    if len(summary_df) == 0:
        south_call = 'Pass'
    else:
        summary_df = summary_df.sort_values("Strength", ascending = False)\
        .sort_values("Count_Trump", ascending = False)\
        .sort_values("Strongest_Card", ascending = False)\
        .sort_values("Non_Trump_Suit_Count", ascending = True)
        best_suit = summary_df[0:1]
        if (best_suit['Count_Trump'].values[0] >= 4 and best_suit['Strongest_Card'].values[0]>=12) \
        or (best_suit['Count_Trump'].values[0] >= 3 and best_suit['Strongest_Card'].values[0]>=12 and ((best_suit['Ace_count'].values[0] + best_suit['King_count'].values[0]) >= 2)):
            trump = best_suit['Suit'].values[0]
            alone_ind = 'Y'
            south_call = 'Order Trump Alone'
            caller = 'south'
        
        elif (best_suit['Count_Trump'].values[0] >= 2 and (best_suit['Right_Bower'].values[0]=='Y') and ((best_suit['Ace_count'].values[0] + best_suit['King_count'].values[0]) >= 2)) \
        or ((best_suit['Count_Trump'].values[0] >= 3) and (best_suit['Right_Bower'].values[0]=='Y')) \
        or (best_suit['Count_Trump'].values[0] >= 4):
            trump = best_suit['Suit'].values[0]
            south_call = 'Order Trump'
            caller = 'south'
        
        else:
            trump = 'none'
            south_call = 'Pass'
            caller = 'none'

    if south_call == 'Pass':
        stuck_ind = 'Y'
        naming = 'south_summary'
        south_df = globals()[naming]
        south_df = south_df.sort_values("Strength", ascending = False)\
        .sort_values("Count_Trump", ascending = False)\
        .sort_values("Strongest_Card", ascending = False)\
        .sort_values("Non_Trump_Suit_Count", ascending = True)
        south_df = south_df.loc[south_df['Suit'] != turned_up['Suit'].values[0]]
        best_suit = south_df[0:1]
        trump = best_suit['Suit'].values[0]
        caller = 'south'

# Function that uses logic to play a game of Euchre. There are a lot of if/else statements each player in each order should generally follow which are encoded
# North, south, east, and west all participate
# Keeps track of the number of tricks north and south wins
def play_cards(order):
    global stats
    global ind_tricks_won
    global team_tricks_won
    stats = south_summary.copy()
    stats = stats.loc[stats['Suit']==trump]
    first = globals()[order[0]]
    second = globals()[order[1]]
    third = globals()[order[2]]
    fourth = globals()[order[3]]
    globals()[f'{order[0]}_play'] = first
    globals()[f'{order[1]}_play'] = second
    globals()[f'{order[2]}_play'] = third
    globals()[f'{order[3]}_play'] = fourth
    
    seen_cards = globals()['seen_cards']
    north_wins = 0
    south_wins = 0
    east_wins = 0
    west_wins = 0
    played = pd.DataFrame(columns = deck.columns.tolist()+['Player'])
    for i in range(5):
        stack = pd.DataFrame(columns = deck.columns.tolist()+['Player'])
        first = globals()[f'{order[0]}_play']
        second = globals()[f'{order[1]}_play']
        third = globals()[f'{order[2]}_play']
        fourth = globals()[f'{order[3]}_play']
        
        first['Player'] = order[0]
        second['Player'] = order[1]
        third['Player'] = order[2]
        fourth['Player'] = order[3]
        first_eval = first.copy()
        second_eval = second.copy()
        third_eval = third.copy()
        fourth_eval = fourth.copy()
        
        first_highest_strength = max(first_eval[f'{trump}_Strength'])
        if (first_eval[f'{trump}_Strength']==13).sum() > 0:
            first_card = first_eval.loc[first_eval[f'{trump}_Strength'] == 13]
        elif (seen_cards[f'{trump}_Strength']>first_highest_strength).sum() == (13 - first_highest_strength) and first_highest_strength >= 7:
            first_card = first_eval.loc[first_eval[f'{trump}_Strength'] == first_highest_strength]
        elif ((seen_cards[f'{trump}_Trump'] == 'Y').sum() >= 4 and first_highest_strength >= 7 and (first_eval[f'{trump}_Strength'] == 6).sum() == 0):
            first_card = first_eval.loc[first_eval[f'{trump}_Strength'] == first_highest_strength]
        elif order[2] == caller and first_highest_strength >= 7 and (first_eval[f'{trump}_Strength'] == 6).sum() == 0:
            first_card = first_eval.loc[first_eval[f'{trump}_Strength'] == first_highest_strength]
        elif (first_eval[f'{trump}_Trump'] == 'N').sum() > 0:
            first_eval = first_eval.loc[first_eval[f'{trump}_Trump'] == 'N'] 
            first_card = first_eval.loc[first_eval[f'{trump}_Strength'] == max(first_eval[f'{trump}_Strength'])].sample(frac=1)[0:1]
        else:
            first_card = first_eval.loc[first_eval[f'{trump}_Strength'] == max(first_eval[f'{trump}_Strength'])].sample(frac=1)[0:1]
        led_suit = first_card[f'{trump}_Suit'].values[0]
            
        strongest_second_trump = max(second_eval[f'{trump}_Strength'])
        if (second_eval[f'{trump}_Suit']==led_suit).sum() > 0:
            best_second_led_suit_val = max(second_eval.loc[second_eval[f'{trump}_Suit']==led_suit][f'{trump}_Strength'])
            worst_second_led_suit_val = min(second_eval.loc[second_eval[f'{trump}_Suit']==led_suit][f'{trump}_Strength'])
            if best_second_led_suit_val > first_card[f'{trump}_Strength'].values[0]:
                second_card = second_eval.loc[(second_eval[f'{trump}_Strength'] == best_second_led_suit_val) & (second_eval[f'{trump}_Suit'] == led_suit)]
            else:
                second_card = second_eval.loc[(second_eval[f'{trump}_Strength'] == worst_second_led_suit_val) & (second_eval[f'{trump}_Suit'] == led_suit)]
        elif ((second_eval[f'{trump}_Strength']>=7) & (second_eval[f'{trump}_Strength'] <= 12 - min((seen_cards[f'{trump}_Strength']>strongest_second_trump).sum(), 2))).sum() > 0:
            second_eval = second_eval.loc[(second_eval[f'{trump}_Strength']>=7) & (second_eval[f'{trump}_Strength'] <= 12 - min((seen_cards[f'{trump}_Strength']>strongest_second_trump).sum(), 2))]
            if (second_eval[f'{trump}_Trump']=='Y').sum() > 1 and (order[1] == caller):
                if len(second_eval.loc[(second_eval[f'{trump}_Strength'] >= 9) & (second_eval[f'{trump}_Strength'] <= 11)]) > 0:
                    second_eval = second_eval.loc[(second_eval[f'{trump}_Strength'] >= 9) & (second_eval[f'{trump}_Strength'] <= 11)]
                second_card = second_eval.loc[second_eval[f'{trump}_Strength'] == min(second_eval[f'{trump}_Strength'])]
            else:
                second_card = second_eval.loc[second_eval[f'{trump}_Strength'] == max(second_eval[f'{trump}_Strength'])]
        else:
            if len(second_eval.loc[(second_eval[f'{trump}_suit_count'] == 1) & (second_eval[f'{trump}_Strength'] <= 4)]) > 0 and (second_eval[f'{trump}_Trump'] == 'Y').sum() > 0:
                second_eval = second_eval.loc[(second_eval[f'{trump}_suit_count'] == 1) & (second_eval[f'{trump}_Strength'] <= 4)]
            second_low = min(second_eval[f'{trump}_Strength'])
            second_card = second_eval.loc[second_eval[f'{trump}_Strength'] == second_low].sample(frac=1)[0:1]
        second_card.loc[(second_card[f'{trump}_Suit'] != led_suit) & (second_card[f'{trump}_Suit'] != trump), f'{trump}_Strength'] = 0
    

        if (third_eval[f'{trump}_Suit']==led_suit).sum() > 0:
            third_eval = third_eval.loc[third_eval[f'{trump}_Suit']==led_suit]
            if first_card[f'{trump}_Strength'].values[0] > second_card[f'{trump}_Strength'].values[0] and max(third_eval[f'{trump}_Strength']) > first_card[f'{trump}_Strength'].values[0]:
                n_greater_cards_hand = (third_eval[f'{trump}_Strength'] > first_card[f'{trump}_Strength'].values[0]).sum()
                n_cards_between = max(third_eval[f'{trump}_Strength']) - first_card[f'{trump}_Strength'].values[0]
                if n_greater_cards_hand == n_cards_between:
                    third_card = third_eval.loc[third_eval[f'{trump}_Strength'] == min(third_eval[f'{trump}_Strength'])]
                else:
                    third_card = third_eval.loc[third_eval[f'{trump}_Strength'] == max(third_eval[f'{trump}_Strength'])]
            elif first_card[f'{trump}_Strength'].values[0] < second_card[f'{trump}_Strength'].values[0] and max(third_eval[f'{trump}_Strength']) >= second_card[f'{trump}_Strength'].values[0]:
                third_card = third_eval.loc[third_eval[f'{trump}_Strength'] == max(third_eval[f'{trump}_Strength'])]
            else:
                third_card = third_eval.loc[third_eval[f'{trump}_Strength'] == min(third_eval[f'{trump}_Strength'])]
        elif (third_eval[f'{trump}_Suit']==trump).sum() > 0 \
        and (first_card[f'{trump}_Strength'].values[0] < second_card[f'{trump}_Strength'].values[0] or first_card[f'{trump}_Strength'].values[0] < 6) \
        and (third_eval[f'{trump}_Strength'] > second_card[f'{trump}_Strength'].values[0]).sum() > 0:
            third_eval = third_eval.loc[(third_eval[f'{trump}_Strength'] > second_card[f'{trump}_Strength'].values[0]) & (third_eval[f'{trump}_Trump'] == 'Y')]
            if len(third_eval.loc[(third_eval[f'{trump}_Strength'] >= 9) & (third_eval[f'{trump}_Strength'] <= 11 - min((seen_cards[f'{trump}_Strength']>max(third_eval[f'{trump}_Strength'])).sum(), 2))])>0 and caller != order[2]:
                    third_eval = third_eval.loc[(third_eval[f'{trump}_Strength']>=9) & (third_eval[f'{trump}_Strength']<=11 - min((seen_cards[f'{trump}_Strength']>max(third_eval[f'{trump}_Strength'])).sum(), 2))]
                    third_card = third_eval.loc[third_eval[f'{trump}_Strength'] == max(third_eval[f'{trump}_Strength'])]
            else:
                third_eval = third_eval.loc[(third_eval[f'{trump}_Strength'] > second_card[f'{trump}_Strength'].values[0]) & (third_eval[f'{trump}_Trump'] == 'Y')]
                third_card = third_eval.loc[third_eval[f'{trump}_Strength'] == min(third_eval[f'{trump}_Strength'])]
        else:
            if len(third_eval.loc[(third_eval[f'{trump}_suit_count'] == 1) & (third_eval[f'{trump}_Strength'] <= 4)]) > 0 and (third_eval[f'{trump}_Trump'] == 'Y').sum() > 0:
                third_eval = third_eval.loc[(third_eval[f'{trump}_suit_count'] == 1) & (third_eval[f'{trump}_Strength'] <= 4)]
            third_low = min(third_eval[f'{trump}_Strength'])
            third_card = third_eval.loc[third_eval[f'{trump}_Strength'] == third_low].sample(frac=1)[0:1]
        third_card.loc[(third_card[f'{trump}_Suit'] != led_suit) & (third_card[f'{trump}_Suit'] != trump), f'{trump}_Strength'] = 0  

        
        if (fourth_eval[f'{trump}_Suit']==led_suit).sum() > 0:
            fourth_eval = fourth_eval.loc[fourth_eval[f'{trump}_Suit']==led_suit]
            if (second_card[f'{trump}_Strength'].values[0] < first_card[f'{trump}_Strength'].values[0] \
                or second_card[f'{trump}_Strength'].values[0] < third_card[f'{trump}_Strength'].values[0]) \
            and ((fourth_eval[f'{trump}_Strength'] > first_card[f'{trump}_Strength'].values[0]).sum() > 0 \
                 and (fourth_eval[f'{trump}_Strength'] > third_card[f'{trump}_Strength'].values[0]).sum() > 0):
                fourth_eval = fourth_eval.loc[(fourth_eval[f'{trump}_Strength'] > first_card[f'{trump}_Strength'].values[0]) & (fourth_eval[f'{trump}_Strength'] > third_card[f'{trump}_Strength'].values[0])]
                fourth_card = fourth_eval.loc[fourth_eval[f'{trump}_Strength'] == min(fourth_eval[f'{trump}_Strength'])]
            else:
                fourth_card = fourth_eval.loc[fourth_eval[f'{trump}_Strength'] == min(fourth_eval[f'{trump}_Strength'])]
        elif (fourth_eval[f'{trump}_Suit']==trump).sum() > 0 \
        and (second_card[f'{trump}_Strength'].values[0] < first_card[f'{trump}_Strength'].values[0] \
             or second_card[f'{trump}_Strength'].values[0] < third_card[f'{trump}_Strength'].values[0]) \
        and ((fourth_eval[f'{trump}_Strength'] > first_card[f'{trump}_Strength'].values[0]).sum() > 0 \
            and (fourth_eval[f'{trump}_Strength'] > third_card[f'{trump}_Strength'].values[0]).sum() > 0):
            fourth_eval = fourth_eval.loc[(fourth_eval[f'{trump}_Strength'] > first_card[f'{trump}_Strength'].values[0]) & (fourth_eval[f'{trump}_Strength'] > third_card[f'{trump}_Strength'].values[0]) & (fourth_eval[f'{trump}_Suit'] == trump)]
            fourth_card = fourth_eval.loc[fourth_eval[f'{trump}_Strength'] == min(fourth_eval[f'{trump}_Strength'])]
        else:
            if len(fourth_eval.loc[(fourth_eval[f'{trump}_suit_count'] == 1) & (fourth_eval[f'{trump}_Strength'] <= 4)]) > 0 and (fourth_eval[f'{trump}_Trump'] == 'Y').sum() > 0:
                fourth_eval = fourth_eval.loc[(fourth_eval[f'{trump}_suit_count'] == 1) & (fourth_eval[f'{trump}_Strength'] <= 4)]
            fourth_low = min(fourth_eval[f'{trump}_Strength'])
            fourth_card = fourth_eval.loc[fourth_eval[f'{trump}_Strength'] == fourth_low].sample(frac=1)[0:1]
        fourth_card.loc[(fourth_card[f'{trump}_Suit'] != led_suit) & (fourth_card[f'{trump}_Suit'] != trump), f'{trump}_Strength'] = 0
        
        first = first.loc[~(first['ID']==first_card['ID'].values[0])]
        second = second.loc[~(second['ID']==second_card['ID'].values[0])]
        third = third.loc[~(third['ID']==third_card['ID'].values[0])]
        fourth = fourth.loc[~(fourth['ID']==fourth_card['ID'].values[0])]
        stack = pd.concat([stack, first_card, second_card, third_card, fourth_card])
        played = pd.concat([played, stack])
        seen_cards = pd.concat([seen_cards, stack])
        seen_cards = seen_cards.drop_duplicates()
        max_card = max(stack[f'{trump}_Strength'])
        winner = stack.loc[stack[f'{trump}_Strength'] == max_card]['Player'].values[0]

        globals()[f'{order[0]}_play'] = first
        globals()[f'{order[1]}_play'] = second
        globals()[f'{order[2]}_play'] = third
        globals()[f'{order[3]}_play'] = fourth

        player_list = [f'{order[0]}_play', f'{order[1]}_play', f'{order[2]}_play', f'{order[3]}_play']
        count_suits(player_list, suits)
        
        if winner == 'north':
            north_wins += 1
            order = ['north', 'east', 'south', 'west']
        elif winner == 'south':
            south_wins += 1
            order = ['south', 'west', 'north', 'east']
        elif winner == 'east':
            east_wins += 1
            order = ['east', 'south', 'west', 'north']
        else:
            west_wins += 1
            order = ['west', 'north', 'east', 'south']

    ind_tricks_won = south_wins
    team_tricks_won = north_wins+south_wins
    
# Function that plays a hand alone. South will play its hand against east and west without north participating
# Keeps track of the number of tricks north and south wins
def go_alone(order):
    global stats
    global alone_ind_tricks_won
    stats = south_summary.copy()
    stats = stats.loc[stats['Suit']==trump]
    order = create_order(dealer)

    if caller == 'north':
        order.remove('south')
    elif caller == 'south':
        order.remove('north')
    elif caller == 'east':
        order.remove('west')
    else:
        order.remove('east')
    
    first = globals()[order[0]]
    second = globals()[order[1]]
    third = globals()[order[2]]
    seen_cards = globals()['seen_cards']
    globals()[f'{order[0]}_play'] = first
    globals()[f'{order[1]}_play'] = second
    globals()[f'{order[2]}_play'] = third
    

    stack = pd.DataFrame(columns = deck.columns.tolist()+['Player'])
    north_wins = 0
    south_wins = 0
    east_wins = 0
    west_wins = 0
    played = pd.DataFrame(columns = deck.columns.tolist()+['Player'])
    for i in range(5):

        stack = pd.DataFrame(columns = deck.columns.tolist()+['Player'])
        first = globals()[f'{order[0]}_play']
        second = globals()[f'{order[1]}_play']
        third = globals()[f'{order[2]}_play']
        
        first['Player'] = order[0]
        second['Player'] = order[1]
        third['Player'] = order[2]
        first_eval = first.copy()
        second_eval = second.copy()
        third_eval = third.copy()

        first_highest_strength = max(first_eval[f'{trump}_Strength'])
        if (first_eval[f'{trump}_Strength']==13).sum() > 0:
            first_card = first_eval.loc[first_eval[f'{trump}_Strength'] == 13]
        elif (seen_cards[f'{trump}_Strength']>first_highest_strength).sum() == (13 - first_highest_strength) and first_highest_strength >= 7:
            first_card = first_eval.loc[first_eval[f'{trump}_Strength'] == first_highest_strength]
        elif ((seen_cards[f'{trump}_Trump'] == 'Y').sum() >= 3 and first_highest_strength >= 7 and (first_eval[f'{trump}_Strength'] == 6).sum() == 0):
            first_card = first_eval.loc[first_eval[f'{trump}_Strength'] == first_highest_strength]
        elif (first_eval[f'{trump}_Trump'] == 'N').sum() > 0:
            first_eval = first_eval.loc[first_eval[f'{trump}_Trump'] == 'N'] 
            first_card = first_eval.loc[first_eval[f'{trump}_Strength'] == max(first_eval[f'{trump}_Strength'])].sample(frac=1)[0:1]
        else:
            first_card = first_eval.loc[first_eval[f'{trump}_Strength'] == max(first_eval[f'{trump}_Strength'])].sample(frac=1)[0:1]
        led_suit = first_card[f'{trump}_Suit'].values[0]
        

        strongest_second_trump = max(second_eval[f'{trump}_Strength'])
        if (second_eval[f'{trump}_Suit'] == led_suit).sum() > 0:
            second_eval = second_eval.loc[second_eval[f'{trump}_Suit'] == led_suit] 
            if max(second_eval[f'{trump}_Strength']) < first_card[f'{trump}_Strength'].values[0]:
                second_card = second_eval.loc[second_eval[f'{trump}_Strength'] == min(second_eval[f'{trump}_Strength'])]
            else:
                second_card = second_eval.loc[second_eval[f'{trump}_Strength'] == max(second_eval[f'{trump}_Strength'])]
        elif ((second_eval[f'{trump}_Strength']>=7) & (second_eval[f'{trump}_Strength'] <= 12 - min((seen_cards[f'{trump}_Strength']>strongest_second_trump).sum(), 2))).sum() > 0 \
        and not (order[0] != caller and order[1] != caller and first_card[f'{trump}_Strength'].values[0] == 6):
            if (order[1] != caller):
                second_eval = second_eval.loc[(second_eval[f'{trump}_Strength']>=7) & (second_eval[f'{trump}_Strength'] <= 12 - min((seen_cards[f'{trump}_Strength']>strongest_second_trump).sum(), 2))]
                if (order[0] == caller):
                    second_card = second_eval.loc[second_eval[f'{trump}_Strength'] == min(second_eval[f'{trump}_Strength'])]
                else:
                    second_card = second_eval.loc[second_eval[f'{trump}_Strength'] == max(second_eval[f'{trump}_Strength'])]
            else:
                if len(second_eval.loc[(second_eval[f'{trump}_Strength']>=9) & (second_eval[f'{trump}_Strength'] <= 11 - min((seen_cards[f'{trump}_Strength']>max(second_eval[f'{trump}_Strength'])).sum(), 2))])>0:
                    second_eval = second_eval.loc[(second_eval[f'{trump}_Strength']>=9) & (second_eval[f'{trump}_Strength'] <= 11 - min((seen_cards[f'{trump}_Strength']>max(second_eval[f'{trump}_Strength'])).sum(), 2))]
                    second_card = second_eval.loc[second_eval[f'{trump}_Strength'] == max(second_eval[f'{trump}_Strength'])]
                else:
                    second_eval = second_eval.loc[(second_eval[f'{trump}_Strength']>=7) & (second_eval[f'{trump}_Strength'] <= 12 - min((seen_cards[f'{trump}_Strength']>strongest_second_trump).sum(), 2))]
                    second_card = second_eval.loc[second_eval[f'{trump}_Strength'] == min(second_eval[f'{trump}_Strength'])]
        else:
            second_low = min(second_eval[f'{trump}_Strength'])
            second_card = second_eval.loc[second_eval[f'{trump}_Strength'] == second_low].sample(frac=1)[0:1]
        second_card.loc[(second_card[f'{trump}_Suit'] != led_suit) & (second_card[f'{trump}_Suit'] != trump), f'{trump}_Strength'] = 0

        if (third_eval[f'{trump}_Suit'] == led_suit).sum() > 0:
            third_eval = third_eval.loc[third_eval[f'{trump}_Suit'] == led_suit]
            if ((third_eval[f'{trump}_Strength'] > first_card[f'{trump}_Strength'].values[0]).sum() > 0 and first_card[f'{trump}_Strength'].values[0] > second_card[f'{trump}_Strength'].values[0] and order[0] == caller):
                third_eval = third_eval.loc[third_eval[f'{trump}_Strength'] > first_card[f'{trump}_Strength'].values[0]]
                third_card = third_eval.loc[third_eval[f'{trump}_Strength'] == min(third_eval[f'{trump}_Strength'])]
            elif ((third_eval[f'{trump}_Strength'] > second_card[f'{trump}_Strength'].values[0]).sum() > 0 and second_card[f'{trump}_Strength'].values[0] > first_card[f'{trump}_Strength'].values[0] and order[1] == caller):
                third_eval = third_eval.loc[third_eval[f'{trump}_Strength'] > second_card[f'{trump}_Strength'].values[0]]
                third_card = third_eval.loc[third_eval[f'{trump}_Strength'] == min(third_eval[f'{trump}_Strength'])]
            elif ((third_eval[f'{trump}_Strength'] > first_card[f'{trump}_Strength'].values[0]).sum() > 0 and (third_eval[f'{trump}_Strength'] > second_card[f'{trump}_Strength'].values[0]).sum() > 0 and order[2] == caller):
                third_eval = third_eval.loc[(third_eval[f'{trump}_Strength'] > first_card[f'{trump}_Strength'].values[0]) & (third_eval[f'{trump}_Strength'] > second_card[f'{trump}_Strength'].values[0])]
                third_card = third_eval.loc[third_eval[f'{trump}_Strength'] == min(third_eval[f'{trump}_Strength'])]
            else:
                third_card = third_eval.loc[third_eval[f'{trump}_Strength'] == min(third_eval[f'{trump}_Strength'])]
        elif (third_eval[f'{trump}_Suit'] == trump).sum() > 0:
            if ((third_eval[f'{trump}_Strength'] > first_card[f'{trump}_Strength'].values[0]).sum() > 0 and (third_eval[f'{trump}_Suit'] == trump).sum() > 0 and first_card[f'{trump}_Strength'].values[0] > second_card[f'{trump}_Strength'].values[0] and order[0] == caller):
                third_eval = third_eval.loc[(third_eval[f'{trump}_Strength'] > first_card[f'{trump}_Strength'].values[0]) & (third_eval[f'{trump}_Suit'] == trump)]
                third_card = third_eval.loc[third_eval[f'{trump}_Strength'] == min(third_eval[f'{trump}_Strength'])]
            elif ((third_eval[f'{trump}_Strength'] > second_card[f'{trump}_Strength'].values[0]).sum() > 0 and (third_eval[f'{trump}_Suit'] == trump).sum() > 0 and second_card[f'{trump}_Strength'].values[0] > first_card[f'{trump}_Strength'].values[0] and order[1] == caller):
                third_eval = third_eval.loc[(third_eval[f'{trump}_Strength'] > second_card[f'{trump}_Strength'].values[0]) & (third_eval[f'{trump}_Suit'] == trump)]
                third_card = third_eval.loc[third_eval[f'{trump}_Strength'] == min(third_eval[f'{trump}_Strength'])]
            elif ((third_eval[f'{trump}_Strength'] > first_card[f'{trump}_Strength'].values[0]).sum() > 0 and (third_eval[f'{trump}_Strength'] > second_card[f'{trump}_Strength'].values[0]).sum() > 0 and (third_eval[f'{trump}_Suit'] == trump).sum() > 0 and order[2] == caller):
                third_eval = third_eval.loc[(third_eval[f'{trump}_Strength'] > first_card[f'{trump}_Strength'].values[0]) & (third_eval[f'{trump}_Strength'] > second_card[f'{trump}_Strength'].values[0]) & (third_eval[f'{trump}_Suit'] == trump)]
                third_card = third_eval.loc[third_eval[f'{trump}_Strength'] == min(third_eval[f'{trump}_Strength'])]
            else:
                third_card = third_eval.loc[third_eval[f'{trump}_Strength'] == min(third_eval[f'{trump}_Strength'])].sample(frac=1)[0:1]
        else:
            third_card = third_eval.loc[third_eval[f'{trump}_Strength'] == min(third_eval[f'{trump}_Strength'])].sample(frac=1)[0:1]
        third_card.loc[(third_card[f'{trump}_Suit'] != led_suit) & (third_card[f'{trump}_Suit'] != trump), f'{trump}_Strength'] = 0
        
        first = first.loc[~(first['ID']==first_card['ID'].values[0])]
        second = second.loc[~(second['ID']==second_card['ID'].values[0])]
        third = third.loc[~(third['ID']==third_card['ID'].values[0])]
        stack = pd.concat([stack, first_card, second_card, third_card])
        played = pd.concat([played, stack])
        seen_cards = pd.concat([seen_cards, stack])
        seen_cards = seen_cards.drop_duplicates()
        max_card = max(stack[f'{trump}_Strength'])
        winner = stack.loc[stack[f'{trump}_Strength'] == max_card]['Player'].values[0]

        globals()[f'{order[0]}_play'] = first
        globals()[f'{order[1]}_play'] = second
        globals()[f'{order[2]}_play'] = third

        player_list = [f'{order[0]}_play', f'{order[1]}_play', f'{order[2]}_play']
        count_suits(player_list, suits)
    
        if winner == 'north':
            north_wins += 1
            order = ['north', 'east', 'south', 'west']
        elif winner == 'south':
            south_wins += 1
            order = ['south', 'west', 'north', 'east']
        elif winner == 'east':
            east_wins += 1
            order = ['east', 'south', 'west', 'north']
        else:
            west_wins += 1
            order = ['west', 'north', 'east', 'south']
        if caller == 'north':
            order.remove('south')
        elif caller == 'south':
            order.remove('north')
        elif caller == 'east':
            order.remove('west')
        else:
            order.remove('east')

    alone_ind_tricks_won = south_wins


# Function that determines if South will order the turned up suit to be trump based off the cards in its hand 
def order_turned_up(south):
    global caller
    global trump
    global alone_ind
    global seen_cards
    global picked_up_ind
    global stuck_ind
    global south_call
    south = globals()[south]
    seen_cards = pd.DataFrame(columns = south.columns.tolist())
    alone_ind = 'N'
    trump = 'none'
    picked_up_ind = 'N'
    stuck_ind = 'N'
    caller = 'none'

    turned_up_suit = turned_up['Suit'].values[0]
    summary_name = 'south_summary'
    summary_df = globals()[summary_name]
    turned_up_suit_summary = summary_df.loc[summary_df['Suit']==turned_up_suit]
    if (turned_up_suit_summary['Count_Trump'].values[0] >= 4 and turned_up_suit_summary['Strongest_Card'].values[0]>=12) \
    or ((turned_up_suit_summary['Count_Trump'].values[0] >= 3 and turned_up_suit_summary['Strongest_Card'].values[0]>=12) \
    and ((turned_up_suit_summary['Ace_count'].values[0] + turned_up_suit_summary['King_count'].values[0]) >= 2)):
        trump = turned_up_suit
        alone_ind = 'Y'
        picked_up_ind = 'Y'
        caller = 'south'
        if (caller == 'south' and dealer == 'north'):
            seen_cards = pd.concat([turned_up])
        pick_it_up(dealer, trump)
        recount_suits(dealer, suits)
        south_call = 'Order Trump Alone'
    elif (turned_up_suit_summary['Count_Trump'].values[0] >= 2 and (turned_up_suit_summary['Right_Bower'].values[0]=='Y') and ((turned_up_suit_summary['Ace_count'].values[0] + turned_up_suit_summary['King_count'].values[0]) >= 2)) \
    or ((turned_up_suit_summary['Count_Trump'].values[0] >= 3) and (turned_up_suit_summary['Right_Bower'].values[0]=='Y')) \
    or (turned_up_suit_summary['Count_Trump'].values[0] >= 4):
        picked_up_ind = 'Y'
        trump = turned_up_suit
        pick_it_up(dealer, trump)
        recount_suits(dealer, suits)
        south_call = 'Order Trump'
        caller = 'south'
        if (caller == 'south' and dealer == 'north'):
            seen_cards = pd.concat([turned_up])
    else:
        south_call = 'Pass'
    if trump == 'none':
        picked_up_ind = 'Y'
        trump = turned_up_suit
        pick_it_up(dealer, trump)
        recount_suits(dealer, suits)
        south_call = 'Pass'
        caller = 'south'
        if (caller == 'south' and dealer == 'north'):
            seen_cards = pd.concat([turned_up])

# Process for bidding trump

For this project, south has the ability to bid trump for a given suit. This is not the case in real life, but for the sake of comparing results, south will have the full option of bidding trump. 

Here is how the default intuition selection method chooses trump:

When a card gets turned up at the beginning of the round, the summary data for that suit gets analyzed using the logic below. 

If that card gets turned down, the strongest suit (the suit with the highest calculated strength is used) and gets analyzed using the same logic below: 

**GO ALONE if the hand satisfies these conditions:**
- There are 4 or more trump cards of a certain suit and the Left or Right Bower is present in hand
- OR There are 3 or more trump cards of a certain suit, the Left or Right Bower is present in hand, and the non-trump cards are either king or ace rank
- AND An opponent will not be picking up a jack


**ORDER TRUMP if the hand satisfies these conditions:**
- There are at least 2 trump cards in hand, the Right Bower is present, and there are at least 2 non-trump cards of king or ace rank
- OR There are at least 3 trump cards with the Right Bower present
- OR There are at least 4 trump cards present


**Otherwise — PASS**

# Process logic for playing cards with all 4 players

**First**
- If the Right Bower is in hand, *play the Right Bower*
- Else if the hand has the expected strongest trump card that hasn’t been played yet, *lead with that card*
- Else if at least 4 trump cards have been played and there are no non-trump aces in hand, but there are trump cards in hand, *lead with the strongest available trump card*
- Else if the partner made the bid and there are no non-trump aces in hand, but there are trump cards in hand, *lead with the strongest available trump card*
- Otherwise, *lead with the strongest available non-trump card*

**Second**
- If the second player has a card of the suit that was led, *they must play it*
    - If the second player has a card stronger than than the first card, *play the strongest available card of the led suit*
    - Otherwise, *play the weakest available card of the led suit*
- If the second player has trump cards (excluding the expected strongest card that hasn’t been played), *play those trump cards*
    - If the second player made the bid and there are mid-value trump cards (queen, king, or ace) in hand, *play the lowest available trump card from that list*
    - Otherwise, *play that highest available trump card that is not the expected strongest card*
- Else *play the weakest card. Ideally, play a lower-value card of a suit that is the only card of that suit in hand*

**Third**
- If the third player has a card of the suit that was led, *they must play it*
    - If the third player has a card of the led suit that is stronger than the first 2 played cards and the partner is not guaranteed to win the trick, *play the strongest available card of the led suit*
    - Otherwise, *play the weakest available card of the led suit*
- Else if the third player has a trump card and the first player’s card is not currently winning or if the first card is not a non-trump ace, *play a trump card (preferably a mid-value trump card if there’s one available, or the weakest available trump card)*
- Else *play the weakest card. Ideally, play a lower-value card of a suit that is the only card of that suit in hand*

**Fourth**
- If the fourth player has a card of the suit that was led, *they must play it*
    - If the fourth player has a card of the led suit that can win the trick that is currently not being won by the fourth player’s team, *play the weakest available card of the led suit that can win the trick*
    - Otherwise, *play the weakest available card of the led suit*
- Else if the fourth player has a trump card that can win a trick that is currently not being won by the fourth player’s team, *play the weakest available trump card that can win the fourth player the trick*
- Else *play the weakest card. Ideally, play a lower-value card of a suit that is the only card of that suit in hand*

# Process logic for playing cards when a player is going alone

**First**
- If the Right Bower is present, *play the Right Bower* (fishes for trump cards and guarantees a free trick)
- Else if all the cards stronger than the strongest trump card in hand have been played, *play the strongest available trump card in hand* (also fishes for trump and guarantees a free trick)
- Else if at least 3 total trump cards have been played and the strongest card in hand is a trump and there are no non-trump aces in hand, *play the strongest available trump card in hand*
- Else *play the strongest non-trump card in hand*

**Second**
- If the second player has a card of the suit that was led, *play a card of that suit*
    - If the second player has a card of the led suit that is stronger than the first card, *play the strongest available card of the led suit*
    - Otherwise, *play the weakest available card of the led suit*
- Else if the second player has trump cards (excluding the expected strongest card), and would not be trumping a partner’s played ace, *play a trump card*
    - If the first player made the bid, win the trick by *playing the weakest available trump card*
    - Otherwise, *play a strong trump card* (but not the expected strongest card)
- Else *play the lowest value card in hand*

**Third**
- If the third player has a card of the suit that was led, *play a card of that suit*
    - If the player has a card of the led suit that can win the trick that otherwise would not be won by the team, *play the weakest available card of the led suit that would win the trick*
    - Otherwise, *play the weakest available card of the led suit*
- Else If the third player has a trump card that would win the trick for the team that otherwise would not be won, *play the weakest available trump card that would win the trick*
- Else *play the lowest value card in hand*

# Create deck of cards

In [3]:
# Create a deck of cards
ID = list(range(1,25))
Value = ['Nine', 'Ten', 'Jack', 'Queen', 'King', 'Ace']*4
Suit = list(np.repeat(['Hearts', 'Diamonds', 'Clubs', 'Spades'], 6))
Color = list(np.repeat(['Red', 'Black'], 12))
deck = pd.DataFrame(list(zip(ID, Value, Suit, Color)), columns = ['ID', 'Value', 'Suit', 'Color'])

# Create separate dataframes that identify the strength values of each card depending on if that suit is trump
Hearts_are_Trump = deck.copy()
Hearts_are_Trump['Strength'] = [7,8,13,9,10,11,1,2,12,4,5,6,1,2,3,4,5,6,1,2,3,4,5,6]
Diamonds_are_Trump = deck.copy()
Diamonds_are_Trump['Strength'] = [1,2,12,4,5,6,7,8,13,9,10,11,1,2,3,4,5,6,1,2,3,4,5,6]
Clubs_are_Trump = deck.copy()
Clubs_are_Trump['Strength'] = [1,2,3,4,5,6,1,2,3,4,5,6,7,8,13,9,10,11,1,2,12,4,5,6]
Spades_are_Trump = deck.copy()
Spades_are_Trump['Strength'] = [1,2,3,4,5,6,1,2,3,4,5,6,1,2,12,4,5,6,7,8,13,9,10,11]

# append the scenarios in which each suit is trump to the main deck dataframe 
deck[['Hearts_Strength', 'Hearts_Bower', 'Hearts_Trump']] = Hearts_are_Trump.apply(create_indicators, axis = 1, result_type = 'expand')
deck[['Diamonds_Strength', 'Diamonds_Bower', 'Diamonds_Trump']] = Diamonds_are_Trump.apply(create_indicators, axis = 1, result_type = 'expand')
deck[['Clubs_Strength', 'Clubs_Bower', 'Clubs_Trump']] = Clubs_are_Trump.apply(create_indicators, axis = 1, result_type = 'expand')
deck[['Spades_Strength', 'Spades_Bower', 'Spades_Trump']] = Spades_are_Trump.apply(create_indicators, axis = 1, result_type = 'expand')

# Define player and suit list
players = ['north', 'south', 'east', 'west']
suits = ['Hearts', 'Diamonds', 'Clubs', 'Spades']

# Change the suit type for a card if it is the Left Bower
alter_suits(suits)

# Identify the first dealer and establish the dealer order cycle
first_dealer = random.sample(players, k=4)[0]
dealer_cycle = create_order(first_dealer)


# Simulate rounds of Euchre

In [4]:
import time
# Find time it takes to run code
t0 = time.time()
# identify number of hands to simulate (100K for training data, 50K for test data)
for i in range(100): # 100 Euchre simulations. Can increase if needed
    dealer = dealer_cycle[i % 4] # identify dealer for round
    order = create_order(dealer) # establish system order for round

    # shuffle created deck
    shuffle = deck.sample(frac=1)
    north = shuffle[0:5]
    south = shuffle[5:10]
    east = shuffle[10:15]
    west = shuffle[15:20]
    turned_up = shuffle[20:21]
    buried = shuffle[21:24]

    # Create column that counts the number of suits present in hand for each row in dataframe
    count_suits(players, suits)
    # Build dataframe for each player that analyzes each suit a player has and has columns describing the hand more in detail
    build_evaluation(players, suits)
    

    
    ########### Create simulations for bidding on a suit that was NOT turned up ############
    
    alone_ind_tricks_won = 0

    # Determine if south would call to pick the turned up card up and identify south's best suit if south did not call to pick up
    south_decide_trump('south')
    seen_cards = pd.concat([turned_up])

    # Play the round of cards
    play_cards(order)
    
    # Evaluate the number of tricks won by south and by south and north combined
    team_ind_tricks_won = ind_tricks_won
    team_tricks_won = team_tricks_won
    # If south wins at least 1 trick and the team (north and south) win at least 3 tricks, attempt for south to go alone
    if team_tricks_won >= 3 and team_ind_tricks_won >= 1:
        correct_call = 'Order Trump'
        go_alone(order)
        alone_ind_tricks_won = alone_ind_tricks_won
        # set the correct call equal to 'go alone' if south wins all 5 tricks or if south wins 4 tricks going alone (without the Right Bower or with 4 trump cards) and north didn't help when playing in the original iteration
        if alone_ind_tricks_won == 5 \
        or (stats['Right_Bower'].values[0] == 'N' and alone_ind_tricks_won == 4 and alone_ind_tricks_won >= team_tricks_won and team_ind_tricks_won >= 3) \
        or (stats['Count_Trump'].values[0] >= 4 and alone_ind_tricks_won == 4 and alone_ind_tricks_won >= team_tricks_won and team_ind_tricks_won >= 3):
            correct_call = 'Order Trump Alone'
        # If south wins 1 trick or less going alone and south wins less than 2 tricks playing the round with north, set the correct call = pass
        elif alone_ind_tricks_won <= 1 and not (team_tricks_won >= 3 and team_ind_tricks_won >= 2):
            correct_call = 'Pass'
    else:
        correct_call = 'Pass'
    # Create a broad start order (determines if south was first, in the middle, or the last to play the first trick)
    if stats['Start_Order'].values[0] == 1:
        stats['Broad_Start_Order'] = 'First'
    elif stats['Start_Order'].values[0] == 4:
        stats['Broad_Start_Order'] = 'Last'
    else:
        stats['Broad_Start_Order'] = 'Middle'
    # Evaluate if south, north, or the opponents (east or west) picked up a card and lists that value
    stats['ind_picked_up'] = 'None'
    stats['p_picked_up'] = 'None'
    stats['opp_picked_up'] = 'None'

    # Adds a summary of the tricks won by south and the team when playing together and south's tricks won when going alone
    stats['team_ind_tricks_won'] = team_ind_tricks_won
    stats['team_tricks_won'] = team_tricks_won
    stats['alone_tricks_won'] = alone_ind_tricks_won
    stats['predicted_call'] = south_call
    stats['correct_call'] = correct_call

    # Appends a summary of south's hand and outcome to a dataframe that stores all the results
    # Create dataframe on the first iteration
    if i == 0:
        global all_stats_make_trump
        all_stats_make_trump = pd.DataFrame(columns = stats.columns.tolist())
        all_stats_make_trump = pd.concat([all_stats_make_trump, stats])
    else:
        # append record to dataframe
        all_stats_make_trump = pd.concat([all_stats_make_trump, stats])

    

    ########### Create simulations for bidding on the turned up card ############
    
    order = create_order(dealer)
    alone_ind_tricks_won = 0
    
    # Determine if south would call to pick the turned up card up and identify south's best suit if south did not call to pick up
    order_turned_up('south')

    #south_decide_trump('south')
    # Play the round of cards
    play_cards(order)
    # evaluate the number of tricks won by south and by south and north combined
    team_ind_tricks_won = ind_tricks_won
    team_tricks_won = team_tricks_won
    # If south wins at least 1 trick and the team (north and south) win at least 3 tricks, attempt for south to go alone
    if team_tricks_won >= 3 and team_ind_tricks_won >= 1:
        correct_call = 'Order Trump'
        go_alone(order)
        alone_ind_tricks_won = alone_ind_tricks_won
        # set the correct call equal to 'go alone' if south wins all 5 tricks or if south wins 4 tricks going alone (without the Right Bower or with 4 trump cards) and north didn't help when playing in the original iteration
        if alone_ind_tricks_won == 5 \
        or (stats['Right_Bower'].values[0] == 'N' and alone_ind_tricks_won == 4 and alone_ind_tricks_won >= team_tricks_won and team_ind_tricks_won >= 3) \
        or (stats['Count_Trump'].values[0] >= 4 and alone_ind_tricks_won == 4 and alone_ind_tricks_won >= team_tricks_won and team_ind_tricks_won >= 3):
            correct_call = 'Order Trump Alone'
        # If south wins 1 trick or less going alone and south wins less than 2 tricks playing the round with north, set the correct call = pass
        elif alone_ind_tricks_won <= 1 and not (team_tricks_won >= 3 and team_ind_tricks_won >= 2):
            correct_call = 'Pass'
    else:
        correct_call = 'Pass'
    # Create a broad start order (determines if south was first, in the middle, or the last to play the first trick)
    if stats['Start_Order'].values[0] == 1:
        stats['Broad_Start_Order'] = 'First'
    elif stats['Start_Order'].values[0] == 4:
        stats['Broad_Start_Order'] = 'Last'
    else:
        stats['Broad_Start_Order'] = 'Middle'
    # Evaluate if south, north, or the opponents (east or west) picked up a card and lists that value
    stats['ind_picked_up'] = 'None'
    stats['p_picked_up'] = 'None'
    stats['opp_picked_up'] = 'None'
    if picked_up_ind == 'Y':
        if dealer == 'south':
            stats['ind_picked_up'] = turned_up['Value'].values[0]
        elif dealer == 'north':
            stats['p_picked_up'] = turned_up['Value'].values[0]
        else:
            stats['opp_picked_up'] = turned_up['Value'].values[0]

    # Add exclusion where opponent picks up a jack since it is never wise to go alone when an opponent picks up a jack. It still is okay to order trump if hand is good enough to
    if correct_call == 'Order Trump Alone' and stats['opp_picked_up'].values[0] == 'Jack':
        correct_call = 'Order Trump'

    if south_call == 'Order Trump Alone' and stats['opp_picked_up'].values[0] == 'Jack':
        south_call = 'Order Trump'
        
    # Adds a summary of the tricks won by south and the team when playing together and south's tricks won when going alone
    stats['team_ind_tricks_won'] = team_ind_tricks_won
    stats['team_tricks_won'] = team_tricks_won
    stats['alone_tricks_won'] = alone_ind_tricks_won
    stats['predicted_call'] = south_call
    stats['correct_call'] = correct_call

    # Appends a summary of south's hand and outcome to a dataframe that stores all the results
    # Create dataframe on the first iteration
    if i == 0:
        global all_stats_turn_up
        all_stats_turn_up = pd.DataFrame(columns = stats.columns.tolist())
        all_stats_turn_up = pd.concat([all_stats_turn_up, stats])
    else:
        # append record to dataframe
        all_stats_turn_up = pd.concat([all_stats_turn_up, stats])

# records time after code executes
t1 = time.time()

# Evaluate the amount of time it took to run code

In [5]:
print(f'code took {round(t1-t0, 0)} seconds to run')

code took 50.0 seconds to run


# Replace Y and N values with 1s and 0s to make binary

In [6]:
all_stats_turn_up = all_stats_turn_up.replace('Y', 1).replace('N', 0)
all_stats_make_trump = all_stats_make_trump.replace('Y', 1).replace('N', 0)

# Write result dataframes with simulations to csv files that get analyzed in R

In [7]:
# To export training data, uncomment the 2 lines below and change number of iterations if necessary and rerun all code from the top

# all_stats_turn_up.to_csv('euchre_turn_up_stats_train.csv', index = False)
# all_stats_make_trump.to_csv('euchre_make_trump_stats_train.csv', index = False)

# To export test data, uncomment the 2 lines below and change number of iterations if necessary and rerun all code from the top

# all_stats_turn_up.to_csv('euchre_turn_up_stats_test.csv', index = False)
# all_stats_make_trump.to_csv('euchre_make_trump_stats_test.csv', index = False)